In [30]:
import math
import pandas as pd
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from datetime import datetime, date, timedelta
from xbbg import blp
import blpapi
import os
import win32com.client as win32
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.io import to_html
from plotly.subplots import make_subplots
import yfinance as yf
import pandas as pd
import numpy as np
import math
import datetime
import statsmodels.api as sm
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import plot
import scipy.optimize as spop
from scipy.stats import t
from scipy.stats import norm
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.colors as mcolors
cmap = mcolors.LinearSegmentedColormap.from_list(
    "soft_RdYlGn", ["lightcoral", "white", "lightgreen"]
)

####### MISC PHOTOS ############
Our_Asset_Class_Views = '<img src="https://i.imgur.com/wMquJZo.png" width="800">'
Fla = '<img src="https://i.imgur.com/xPi0AgA.png" width="800">'
Funda = '<img src="https://i.imgur.com/fYmUJ4d.png" width="800">'
Trad = '<img src="https://i.imgur.com/5hqqFJP.png" width="800">'
Cfs_fc = '<img src="https://i.imgur.com/Nh18Zhx.png" width="800">'
basecase = '<img src="https://i.imgur.com/mWuqeJA.png" width="700">'
cape_chart = '<img src="https://i.imgur.com/RsszGsQ.png" width="700">'
concentration = '<img src="https://i.imgur.com/JJQxKdV.png" width="700">'
eco_surprise = '<img src="https://i.imgur.com/mdfxHrD.png" width="700">'
recession = '<img src="https://i.imgur.com/6zODN1y.png" width="700">'
sys_outputs = '<img src="https://i.imgur.com/ENC01In.png" width="700">'
region_positioning = '<img src="https://i.imgur.com/VFEfrIh.png" width="700">'
returns_2022 = '<img src="https://i.imgur.com/e6xcaLa.png" width="700">' 
crsp_dimensional = '<img src="https://i.imgur.com/Rz9Kqrz.png" width="800">'
deficit_matrix = '<img src="https://imgur.com/0e160541-a5fc-4e44-9543-dd2453b4f153" width="800">'
tariff_2025 = '<img src="https://i.imgur.com/ZKUHcjO.png" width="800">'
tariff_2025_asia = '<img src="https://i.imgur.com/T9y1j74.png" width="800">'

# Scrape from Nowcast
today_date = datetime.now().strftime("%B %d, %Y")
url = "https://www.atlantafed.org/cqer/research/gdpnow"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

page_text = soup.get_text()
keyword = "Latest estimate:"
pos = page_text.find(keyword)

if pos != -1:
    start_index = pos + len(keyword)
    # Safely get the next 24 characters (or fewer if the string is short)
    next_24_chars = page_text[start_index : start_index + 24]
    print("Latest estimate:", next_24_chars)
else:
    print("No 'Latest estimate:' found in the text.")



def winsorize_df(df, lower_limit=0.01, upper_limit=0.99):
    return df.apply(lambda col: winsorize(col, limits=(lower_limit, 1-upper_limit)))

def timeseriesplotting(valuation_df, valuation_metric, country):
    country_df = valuation_df[country]
    data = country_df[valuation_metric].dropna()  
    mean_val = data.median()
    std_val = data.std()
    fig = px.line(data, width = 1300, height = 600, title = f'{valuation_metric} : {country}')
    fig.update_traces(line=dict(color='#30415f'))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val] * len(data), mode='lines', name='Mean', line=dict(dash='solid', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val + std_val] * len(data), mode='lines', name='+ 1 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val - std_val] * len(data), mode='lines', name='- 1 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val - (std_val*2)] * len(data), mode='lines', name='- 2 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val + (std_val*2)] * len(data), mode='lines', name='+ 1 Std', line=dict(dash='dot', color = 'grey')))

    fig.update_layout(font=dict(family="Montserrat", size=13), title=dict(font=dict(family="Montserrat", size=16)), plot_bgcolor='white')
    
    valuation_list = ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']
    
    if valuation_metric == valuation_list[0]:
        fig.update_layout(title=dict(text=f'<b><span style="color:darkcyan;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[1]:
        fig.update_layout(title=dict(text=f'<b><span style="color:coral;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[2]:
        fig.update_layout(title=dict(text=f'<b><span style="color:crimson;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[3]:
        fig.update_layout(title=dict(text=f'<b><span style="color:cadetblue;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))

    fig.update_xaxes(tickangle=45, title_text="", tickfont=dict(size=10))
    fig.update_yaxes(title_text=f'{valuation_metric}')

    return fig

def per_valuation_plotter(name_list, valuation_list, valuation_metric, winsorized_add_composite):
    total_plots = len(name_list)
    rows = math.ceil(total_plots / 2)  # Round up to handle odd numbers

    # Create a subplot figure with the specified rows and 2 columns
    fig = make_subplots(rows=rows, cols=3, subplot_titles=[f"{valuation_metric} - {j}" for j in name_list])

    # Loop over each region and add the corresponding plot to the subplot
    for idx, j in enumerate(name_list):
        row = (idx // 3) + 1  # Calculate the row
        col = (idx % 3) + 1   # Calculate the column

        # Generate the plot for the region with the specified valuation metric
        initial = timeseriesplotting(winsorized_add_composite, valuation_metric, j)
        
        # Add each trace in `initial` to the subplot, hiding the legend
        for trace in initial.data:
            trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)

        # Apply individual title and font color based on `valuation_metric` for each subplot
        title_color = 'black'  # Default color
        if valuation_metric == valuation_list[0]:
            title_color = 'darkcyan'
        elif valuation_metric == valuation_list[1]:
            title_color = 'coral'
        elif valuation_metric == valuation_list[2]:
            title_color = 'crimson'
        elif valuation_metric == valuation_list[3]:
            title_color = 'cadetblue'
        elif valuation_metric == valuation_list[4]:
            title_color = 'purple'
        elif valuation_metric == valuation_list[5]:
            title_color = 'green'

        # Update title for each subplot individually
        fig.layout.annotations[idx].update(
            text=f'<b><span style="color:{title_color};">{valuation_metric}</span> : {j}</b>',
            font=dict(size=16, family='Montserrat')
        )

    # Update layout with global settings for font and plot background color
    fig.update_layout(
        height=400 * rows, width=1500,
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white'
    )

    # Apply X and Y axis formatting to all subplots
    fig.update_xaxes(tickangle=45, title_text="", tickfont=dict(size=10))
    fig.update_yaxes(title_text=f'{valuation_metric}')

    return fig

def clean_data_after_bloomberg(multi_level_df, names_of_the_category, valuation_list):
    multi_level_df.columns = multi_level_df.columns.set_levels([names_of_the_category, valuation_list])
    multi_level_df.index = pd.to_datetime(multi_level_df.index)
    multi_level_df.index = multi_level_df.index.strftime('%b-%Y')
    multi_level_df_winsorized = winsorize_df(multi_level_df, 0.01, 0.99)

    for i in multi_level_df_winsorized.columns.levels[0]:
        country = multi_level_df_winsorized[i]
        z_score_per_val = (( country - country.mean() ) / country.std() )   # seperate Z_score DF
        without_PE = z_score_per_val.drop('Forward PE', axis = 1)       
        valuation_z_score = without_PE.mean(axis=1)                          # Z-score w/o F-PE
        multi_level_df_winsorized[(i,'Valuation Composite')] = valuation_z_score            # add that main
    multi_level_df_winsorized_add_composite = multi_level_df_winsorized.replace(0, np.nan)
    
    # if names_of_the_category == regionals_name:
    #     valuation_list.append('Valuation Composite')
    valuation_list.append('Valuation Composite')

    return multi_level_df_winsorized_add_composite, valuation_list, z_score_per_val

    # Set MultiIndex column levels: (Region, Metric)
    multi_level_df.columns = pd.MultiIndex.from_product([names_of_the_category, valuation_list])
    multi_level_df.index = pd.to_datetime(multi_level_df.index)
    multi_level_df.index = multi_level_df.index.strftime('%b-%Y')

    df_winsor = winsorize_df(multi_level_df.copy(), 0.01, 0.99)

    for name in names_of_the_category:
        sub_df = df_winsor[name]
        z_score_df = (sub_df - sub_df.mean()) / sub_df.std()
        z_score_df = z_score_df.drop(columns=['Forward PE'], errors='ignore')
        composite = z_score_df.mean(axis=1)
        df_winsor[(name, 'Valuation Composite')] = composite

    valuation_list.append('Valuation Composite')

    return df_winsor.replace(0, np.nan), valuation_list, z_score_df

def generate_graphs(values_composite, names, prefix):
    for valuation_metric in values_composite[1]:
        var_name = f"{prefix}_{valuation_metric.replace(' ', '_').replace('/', '_')}"  # Fix naming
        graph_list = [plot(timeseriesplotting(values_composite[0], valuation_metric, name), 
                           output_type='div', include_plotlyjs='cdn') for name in names]

        globals()[var_name] = graph_list  # Directly create variables
        print(f"{var_name} = {graph_list}")  # Print for visibility

def graph_performance(data, title):
    # Color palette from full graph_performance
    full_palette = [
        "#30415f", "#f3a712", "#87b1a1", "#5ac5fe",
        "#a8c686", "#a0a197", "#e4572e", "#2337C6",
        "#B7B1B0", "#778BA5", "#990000"
    ]

    # Simpler palette from simp_graph_performance for ≤ 4 lines
    simp_palette = [
        "#30415f", "#DDDDDD", "#DDDDDD", "#DDDDDD"
    ]

    # Determine color usage
    if isinstance(data, pd.Series):
        palette = full_palette
        fig = px.line(data, width=1850, color_discrete_sequence=palette)

    elif isinstance(data, pd.DataFrame):
        use_full_colors = data.shape[1] > 4
        palette = full_palette if use_full_colors else simp_palette
        fig = px.line(data, width=1850 if use_full_colors else 1250)

        for i, trace in enumerate(fig.data):
            trace.update(line=dict(color=palette[i % len(palette)]))

    # Layout setup (consistent)
    fig.update_layout(
        font_family="Montserrat",
        title={"text": title, "font": {"size": 22}},
        xaxis_title="",
        yaxis_title="Price",
        yaxis=dict(
            side="left",
            title="Price",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            gridcolor="#ECECEC",
            linecolor="#ECECEC",
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        height=600,
        legend=dict(
            orientation="h",
            y=-0.075,
            x=0.5,
            xanchor="center"
        )
    )

    return plot(fig, output_type='div', include_plotlyjs='cdn')


def simp_graph_performance(data, title):
    # Color palette from full graph_performance
    full_palette = [
        "#30415f", "#f3a712", "#87b1a1", "#5ac5fe",
        "#a8c686", "#a0a197", "#e4572e", "#2337C6",
        "#B7B1B0", "#778BA5", "#990000"
    ]

    # Simpler palette from simp_graph_performance for ≤ 4 lines
    simp_palette = [
        "#30415f", "#DDDDDD", "#DDDDDD", "#DDDDDD"
    ]

    # Determine color usage
    if isinstance(data, pd.Series):
        palette = full_palette
        fig = px.line(data, width=1850, color_discrete_sequence=palette)

    elif isinstance(data, pd.DataFrame):
        use_full_colors = data.shape[1] > 4
        palette = full_palette if use_full_colors else simp_palette
        fig = px.line(data, width=1850 if use_full_colors else 1250)

        for i, trace in enumerate(fig.data):
            trace.update(line=dict(color=palette[i % len(palette)]))

    # Layout setup (consistent)
    fig.update_layout(
        font_family="Montserrat",
        title={"text": title, "font": {"size": 22}},
        xaxis_title="",
        yaxis_title="Price",
        yaxis=dict(
            side="left",
            title="Price",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            gridcolor="#ECECEC",
            linecolor="#ECECEC",
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        height=600,
        legend=dict(
            orientation="h",
            y=-0.075,
            x=0.5,
            xanchor="center"
        )
    )

    return plot(fig, output_type='div', include_plotlyjs='cdn')

def cross_sectional_current_table_maker(factor_vals_winsorized_add_composite):
    """
    Creates an HTML table styled like the existing dashboard tables from a DataFrame.
    Replaces the Plotly table generation.
    """
    if factor_vals_winsorized_add_composite[0].columns.levels[0][0] == 'Value':
        xs_factor_current = factor_vals_winsorized_add_composite[0].iloc[-1].reset_index().pivot(
            index='level_1',  # Set 'level_1' as the rows
            columns='level_0',  # Set 'level_0' as the columns
            values=factor_vals_winsorized_add_composite[0].iloc[-1].name
        ).T.reset_index().rename({'level_0': 'Global Factors'}, axis=1).round(1).sort_values('Forward PE', ascending=False)
    else:
        xs_factor_current = factor_vals_winsorized_add_composite[0].iloc[-1].reset_index().pivot(
            index='level_1',  # Set 'level_1' as the rows
            columns='level_0',  # Set 'level_0' as the columns
            values=factor_vals_winsorized_add_composite[0].iloc[-1].name
        ).T.reset_index().rename({'level_0': 'Region'}, axis=1).round(1).sort_values('Forward PE', ascending=False)

    df = xs_factor_current.copy() # Create a copy to avoid modifying the original

    # Define styles for the table
    styles = [
        {'selector': 'td:hover', 'props': [('background-color', '#30415f')]},
        {'selector': 'th:not(.index_name)', 'props': [('background-color', '#30415f'), ('color', 'white'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('font-size', '14px'), ('text-align', 'center'), ('width', '120px'), ('border', '1px solid #ddd')]},
        {'selector': 'th', 'props': [('text-align', 'left'), ('border', '1px solid #ddd')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse')]}
    ]

    # Add color gradient to Valuation Composite column
    styled_table = (df.style
        .set_table_styles(styles)
        .format(precision=1)
        .background_gradient(
            subset=['Valuation Composite'],  # Target specific column
            cmap='RdYlGn_r',  # Red-Yellow-Green reversed (green high, red low)
            axis=0  # Apply per-column
        )
        .hide_index()
        .to_html())

    return styled_table

def winsorize_df(df, lower, upper):
    return df.apply(lambda x: np.clip(x, np.percentile(x.dropna(), lower * 100), np.percentile(x.dropna(), upper * 100)) if np.issubdtype(x.dtype, np.number) else x)

def process_index(raw_df, lower=0.01, upper=0.99):
    # Winsorize and calculate stats
    winsorized = winsorize_df(raw_df, lower, upper)
    
    # Calculate column-wise statistics
    medians = winsorized.median()
    stds = winsorized.std()
    
    # Normalize using column stats
    normalized = (winsorized - medians) / stds
    mean_series = normalized.mean(axis=1)
    
    # Create final DataFrame with bounds
    return pd.DataFrame({
        'z_score': mean_series,
        'mean': mean_series.mean(),  # Average of column medians
        'upper_bound': mean_series+1 ,
        'lower_bound': mean_series-1
    })

def calc_return_1m_to_10y(sector_df, sector_names, cmap="RdYlGn"):
    sector_df.columns = sector_names
    onemonth = (sector_df.iloc[-1] / sector_df.iloc[-2]) - 1
    threemonth = (sector_df.iloc[-1] / sector_df.iloc[-4]) - 1
    sixmonth = (sector_df.iloc[-1] / sector_df.iloc[-7]) - 1
    oneyr = (sector_df.iloc[-1] / sector_df.iloc[-13]) - 1
    threeyr = ((sector_df.iloc[-1] / sector_df.iloc[-37]) ** (1/3)) - 1
    fiveyr = ((sector_df.iloc[-1] / sector_df.iloc[-61]) ** (1/5)) - 1
    tenyr = ((sector_df.iloc[-1] / sector_df.iloc[-121]) ** (1/10)) - 1

    returns_dict = {
        '1 Month': onemonth,
        '3 Months': threemonth,
        '6 Months': sixmonth,
        '1 Year': oneyr,
        '3 Years': threeyr,
        '5 Years': fiveyr,
        '10 Years': tenyr
    }

    returns_df = pd.DataFrame(returns_dict, index=sector_df.columns).applymap(lambda x: f"{x*100:.2f}%")
    return returns_df

Latest estimate:  -2.4 percent — April 09


Sometimes need to re-execute vals:

In [ ]:
########################
########################
########################
########################
# VALUATION , MONTHLY STUFF
########################

class ValuationAnalytics:
    def __init__(self, blp):
        self.blp = blp

        # Core configs
        self.frequency = 'M'
        self.start_date = '1997-05-10'
        self.lower = 0.01
        self.upper = 0.99

        # Valuation metrics and labels
        self.valuation_metric_list = ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO']
        self.valuation_list = ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']

        # Regional market tickers and names
        self.regional_list = ['MXWO Index','SPX Index','SPW Index', 'NKY Index','AS51 Index','AS38 Index','MXEF Index','SXXP Index','MXKR Index','MXCN Index','UKX Index', 'MXWOU Index']
        self.regionals_name = ['World','US','US Equal Weight', 'Japan','Australia','Aussie Smalls', 'EM','Europe','Korea','China','UK', 'World ex US']

        # Factor index tickers and names
        self.factors_list = ['MXWO000V Index', 'M1WOEV Index', 'MXWO000G Index', 'M1WOQU Index','MXWOSC Index','MXWOLC Index','M1WOMVOL Index']
        self.factors_names = ['Value', 'Enhanced Value', 'Growth', 'Quality', 'Small', 'Large', 'Low Volatility']

    def get_monthly_valuation_data(self):
        raw = self.blp.bdh(tickers=self.regional_list, flds=self.valuation_metric_list, start_date=self.start_date, Per=self.frequency)
        return clean_data_after_bloomberg(raw, self.regionals_name, self.valuation_list)

    def get_factor_valuation_data(self):
        raw = self.blp.bdh(tickers=self.factors_list, flds=self.valuation_metric_list, start_date=self.start_date, Per=self.frequency)
        return clean_data_after_bloomberg(raw, self.factors_names, self.valuation_list)

    def get_region_factor_graphs(self, regional_data, factor_data):
        generate_graphs(regional_data, self.regionals_name, "reig_graphs")
        generate_graphs(factor_data, self.factors_names, "factor_graphs")
        
        output = {
            'regional': {
                metric: "".join(globals()[f"reig_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'EV_Trailing_EBITDA', 'Price_to_Sales', 'Valuation_Composite']
            },
            'factor': {
                metric: "".join(globals()[f"factor_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'EV_Trailing_EBITDA', 'Price_to_Sales', 'Valuation_Composite']
            }
        }
        return output

    def cross_sectional_time_series(self, reg_data, fac_data):
        long = ['Value', 'Small', 'EM', 'World ex US', 'US Equal Weight']
        short = ['Growth', 'Large', 'World', 'US', 'US']
        metric = 'Valuation Composite'

        xs_df = pd.DataFrame({
            'Global Value / Growth': fac_data[0][long[0]][metric] - fac_data[0][short[0]][metric],
            'Small / Large': fac_data[0][long[1]][metric] - fac_data[0][short[1]][metric],
            'EM / DM': reg_data[0][long[2]][metric] - reg_data[0][short[2]][metric],
            'Intl ex US / US': reg_data[0][long[3]][metric] - reg_data[0][short[3]][metric],
            'S&P Equal / S&P 500': reg_data[0][long[4]][metric] - reg_data[0][short[4]][metric]
        }, index=fac_data[0].index)

        return graph_performance(xs_df, 'Cross-Sectional Valuation Composite Spreads')

    def cross_sectional_current_tables(self, reg_data, fac_data):
        return {
            'factor_table': cross_sectional_current_table_maker(fac_data),
            'regional_table': cross_sectional_current_table_maker(reg_data)
        }

    def get_aussie_valuations(self):
        tickers = ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS51 Index']
        labels = ['Forward PE', 'Price to Book', 'CAPE', 'Price to Sales']
        fields = ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'INDX_PX_SALES']
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        raw = self.blp.bdh(tickers=tickers, flds=fields, start_date='2003-05-10', Per='M')
        return clean_data_after_bloomberg(raw, names, labels)

    def get_aussie_graphs(self, data):
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        generate_graphs(data, names, "aus_reig_graphs")
        return {
            metric: "".join(globals()[f"aus_reig_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'Price_to_Sales', 'Valuation_Composite']
        }

    def get_aussie_earnings_revision(self, one_year_ago):
        tickers = ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS52 Index']
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        fwd = self.blp.bdh(tickers=tickers, flds=['INDX_WEIGHTED_EST_ERN'], start_date=one_year_ago, BEST_FPERIOD_OVERRIDE='BF').droplevel(1, axis=1)
        realized = self.blp.bdh(tickers=tickers, flds=['t12_eps_aggte'], start_date=one_year_ago).droplevel(1, axis=1)
        fwd.columns = names
        realized.columns = names

        rev_table = (((fwd / realized) - 1) * 100).round(2)
        return graph_performance(rev_table, 'Blended 4Q Forward Earnings Revisions (%)')

va = ValuationAnalytics(blp)

# Auto-uses preloaded tickers and labels
regional_data = va.get_monthly_valuation_data()
factor_data = va.get_factor_valuation_data()
graphs = va.get_region_factor_graphs(regional_data, factor_data)
xs_graph = va.cross_sectional_time_series(regional_data, factor_data)
aussie_data = va.get_aussie_valuations()
aus_graphs = va.get_aussie_graphs(aussie_data)
# Get one year ago from today
one_year_ago = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
aussie_blend4q_earnings_rev = va.get_aussie_earnings_revision(one_year_ago)
blend4q_earnings_rev = model.get_earnings_revisions()
tables = va.cross_sectional_current_tables(regional_data, factor_data)
regional_val_table = tables['regional_table']
factor_val_table = tables['factor_table']

In [ ]:
########################
########################
########################
# JB TIPS, FUTURES
########################

import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo
from typing import List
import pandas as pd

class BloombergCharts:
    def __init__(self, blp):
        self.blp = blp
        self.font = "Montserrat"

    def fetch_data(self, tickers: List[str], fields: List[str], start_date: str, per: str = None, drop_level=True):
      # Fallback to daily if no periodicity is specified
      per = per or 'D'  # Default to 'D' if None or empty
      df = self.blp.bdh(tickers=tickers, flds=fields, start_date=start_date, Per=per)
      return df.droplevel(axis=1, level=1) if drop_level else df


    def build_rate_futures_chart(self) -> str:
        tickers = ['US0ANM DEC2025 Index', 'AU0ANM DEC2025 Index', 'EZ0BNM DEC2025 Index']
        country_labels = ['US', 'AU', 'EU']
        colors = ["#020035", "#4682B4", "orange"]

        data = self.fetch_data(tickers, ['px_last'], '31-01-2024')
        data.columns = country_labels

        fig = px.line(-data.iloc[20:], color_discrete_sequence=colors)

        fig.update_layout(
            font_family=self.font,
            title={"text": "Number of Rate Cuts by December 2025", "font": {"size": 22}},
            xaxis_title="",
            yaxis_title="Number of Cuts/Hikes",
            yaxis=dict(side="right", titlefont=dict(color="black"), tickfont=dict(color="black"),
                       gridcolor="lightgray", linecolor="gray"),
            xaxis=dict(gridcolor="lightgray", linecolor="gray"),
            plot_bgcolor="white",
            paper_bgcolor="white",
            width=950,
            height=600,
            legend=dict(orientation="h", y=-0.075, x=0.5, xanchor="center")
        )

        return pyo.plot(fig, output_type='div', include_plotlyjs='cdn')

    def build_yield_decomp_chart(self) -> str:
        tickers = ['GTII10 Govt', 'USGG10YR Index', 'USGGBE10 Index']
        df = self.fetch_data(tickers, ['px_last'], '1-11-2014')
        df.columns = ['US Real Yld 10Y', 'US Nominal Yld 10Y', 'US 10Y Breakeven Inflation']

        fig = go.Figure()

        # Stacked: Breakeven Inflation
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df['US 10Y Breakeven Inflation'],
            mode='lines',
            fill='tozeroy',
            name='10Y Breakeven',
            line=dict(color='#a0a197', width=0.5),
            stackgroup='stack'
        ))

        # Stacked: Real Yield
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df['US Real Yld 10Y'],
            mode='lines',
            fill='tonexty',
            name='10Y Real Yield',
            line=dict(color='#30415f', width=0.5),
            stackgroup='stack'
        ))

        # Top Line: Nominal Yield
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df['US Nominal Yld 10Y'],
            mode='lines',
            name='10Y Nominal Yield',
            line=dict(color='#30415f', width=3),
        ))

        fig.update_layout(
            width=1200,
            height=700,
            title=dict(text='US 10Y Yields: Real, Nominal and Breakeven Inflation',
                       font=dict(size=24), x=0.5, xanchor='center'),
            legend=dict(orientation="h", yanchor="bottom", y=-0.2,
                        xanchor="center", x=0.5, font=dict(size=14)),
            xaxis_title="Date",
            yaxis_title="Yield (%)",
            template="plotly_white",
            font=dict(family=self.font),
            yaxis=dict(rangemode="tozero"),
            hovermode="x unified",
        )

        return pyo.plot(fig, output_type='div', include_plotlyjs='cdn')

    def build_tips_chart(self) -> str:
        tickers = ['GTII10 Govt', 'GTGBPII10Y Govt', 'GTAUDII10Y Govt']
        df = self.fetch_data(tickers, ['px_last'], '1-08-2007', per='W')
        df.columns = ['US', 'UK', 'AU']
        return graph_performance(df, "Inflation Linked Bonds")  # Assumes external helper

chart_builder = BloombergCharts(blp)
rate_futures_html = chart_builder.build_rate_futures_chart()
ten_10y_decomp_html = chart_builder.build_yield_decomp_chart()
tips_html = chart_builder.build_tips_chart()

########################
########################
########################
########################
# MAX YIELDS
########################
import plotly.io as pio
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta, date

class YieldCurveVisualizer:
    """Class for generating and visualizing yield curves and spread charts"""
    
    # Class constants
    TICKERS = {
        'US': ['USGG3M Index', 'USGG6M Index', 'USGG12M Index', 'USGG2YR Index', 'USGG3Y Index', 
               'USGG5YR Index', 'USGG7Y Index', 'USGG10Y Index', 'USGG20Y Index', 'USGG30Y Index'],
        'EU': ['GTEUR3M Govt', 'GTEUR6M Govt', 'GTEUR1Y Govt', 'GTEUR2Y Govt', 'GTEUR3Y Govt',
               'GTEUR5Y Govt', 'GTEUR7Y Govt', 'GTEUR10Y Govt', 'GTEUR20Y Govt', 'GTEUR30Y Govt'],
        'AU': ['GTAUD3M Govt', 'GTAUD1Y Govt', 'GTAUD2Y Govt', 'GTAUD3Y Govt', 'GTAUD5Y Govt',
               'GTAUD7Y Govt', 'GTAUD10Y Govt', 'GTAUD20Y Govt', 'GTAUD30Y Govt']
    }
    
    MATURITY_LABELS = {
        'US': ['3m', '6m', '12m', '2y', '3y', '5y', '7y', '10y', '20y', '30y'],
        'EU': ['3m', '6m', '12m', '2y', '3y', '5y', '7y', '10y', '20y', '30y'],
        'AU': ['3m', '12m', '2y', '3y', '5y', '7y', '10y', '20y', '30y']
    }
    
    SPREAD_INDICES = {
        'Aus_comp': {'ticker': 'BACM0 Index', 'field': 'OAS_SPREAD_MID', 'title': 'Ausbond Composite 0+Yr Spread'},
        'Aus_cred': {'ticker': 'BACR0 Index', 'field': 'OAS_SPREAD_MID', 'title': 'Ausbond Credit 0+Yr Spread'},
        'Aus_FRN': {'ticker': 'BAFRN0 Index', 'field': 'OAS_SPREAD_MID', 'title': 'Ausbond Credit FRN 0+Yr Spread'},
        'US_corp': {'ticker': 'LUACOAS Index', 'field': 'PX_LAST', 'title': 'US Agg Corporate OAS Spread'},
        'US_cred': {'ticker': 'LUCROAS Index', 'field': 'PX_LAST', 'title': 'US Agg Credit OAS Spread'}
    }
    
    def __init__(self, blp_client):
        """Initialize with Bloomberg client"""
        self.blp = blp_client
        self.today = datetime.today()
        self.charts = {}
        self.time_periods = {
            'Today': self.today,
            'Three Days': self.today - timedelta(days=3),
            'Week': self.today - timedelta(days=7),
            'Month': self.today - timedelta(days=30)
        }
        
        # Store curve data
        self.curves = {}
        self.global_curves = {}
        self.hedged_curves = {}
    
    def create_line_chart(self, df_long, x_col, y_col, color_col, title, width=900, height=500, dash_styles=None):
        """Create a Plotly line chart with consistent styling"""
        fig = px.line(df_long, x=x_col, y=y_col, color=color_col, title=title, width=width, height=height)
        
        # Apply default styling
        fig.update_traces(line=dict(color='#30415f'))
        
        # Apply dash styles to non-main lines if provided
        if dash_styles:
            main_line = df_long[color_col].unique()[0]
            for i, trace in enumerate(fig.data):
                if trace.name != main_line:
                    trace.line.dash = dash_styles[i % len(dash_styles)] if dash_styles else 'solid'
                    trace.line.color = 'lightgray'
        
        fig.update_layout(
            font=dict(family="Montserrat"),
            plot_bgcolor='white',
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False)
        )
        return fig
    
    def fetch_yield_curve_data(self, region):
        """Fetch yield curve data for a region across different time periods"""
        curves = {}
        for period_name, period_date in self.time_periods.items():
            data = self.blp.bdh(tickers=self.TICKERS[region], flds=['PX_LAST'], 
                          start_date='2022-01-01', end_date=period_date).tail(1)
            curves[period_name] = data
        
        # Combine all periods into one dataframe
        df = pd.concat(curves.values(), axis=0)
        df.columns = df.columns.droplevel(1)
        df = df.rename(index={df.index[i]: period for i, period in enumerate(self.time_periods.keys())})
        
        # Rename columns to maturity labels
        rename_dict = {df.columns[i]: self.MATURITY_LABELS[region][i] for i in range(len(df.columns))}
        df = df.rename(columns=rename_dict)
        
        # Special case for AU: add 6m value
        if region == 'AU' and '6m' not in df.columns:
            df.insert(1, '6m', (df['3m'] + df['12m']) / 2)
        
        return df
    
    def process_spread_data(self, index_name):
        """Process spread data with mean and standard deviation"""
        index_info = self.SPREAD_INDICES[index_name]
        ticker = index_info['ticker']
        field = index_info['field']
        
        data = self.blp.bdh(tickers=ticker, flds=[field], 
                      start_date='2015-01-01', end_date=self.today)
        data.columns = data.columns.droplevel(1)
        
        average = data[ticker].mean()
        stddev = data[ticker].std()
        
        data['Average'] = average
        data['+1 STD'] = average + stddev
        data['-1 STD'] = average - stddev
        
        df = data.reset_index().rename(columns={'index': 'Date'})
        df_long = df.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
        
        return df_long
    
    def generate_individual_curves(self):
        """Generate individual yield curve charts for US, EU, and AU"""
        dash_styles = ['solid', 'dash', 'dash', 'dash']
        
        for region in ['US', 'EU', 'AU']:
            # Fetch and store curve data
            curve_data = self.fetch_yield_curve_data(region)
            self.curves[region] = curve_data
            
            # Transform data for plotting
            curve_chart = curve_data.transpose()
            df = curve_chart.reset_index().rename(columns={'index': 'Maturity'})
            df_long = df.melt(id_vars='Maturity', var_name='TimeFrame', value_name='Yield')
            
            # Create and store chart
            fig = self.create_line_chart(
                df_long, 'Maturity', 'Yield', 'TimeFrame', 
                f'{region} Yield Curve {date.today()}', dash_styles=dash_styles
            )
            self.charts[f'{region}_chart_html'] = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    def generate_spread_charts(self):
        """Generate all spread charts"""
        dash_styles = ['solid', 'dash', 'dash', 'dash']
        
        for index_name, index_info in self.SPREAD_INDICES.items():
            df_long = self.process_spread_data(index_name)
            fig = self.create_line_chart(
                df_long, 'Date', 'OAS_Spread', 'Legend', 
                f"{index_info['title']} {date.today()}", dash_styles=dash_styles
            )
            self.charts[f'{index_name}_chart_html'] = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    def fetch_global_curves(self):
        """Fetch current yield curve data for global comparison"""
        for region in ['US', 'EU', 'AU']:
            data = self.blp.bdh(tickers=self.TICKERS[region], flds=['PX_LAST'], 
                          start_date='2022-01-01', end_date=self.today).tail(1)
            data = data.rename(index={data.index[0]: f'{region} Yield Curve'})
            data.columns = data.columns.droplevel(1)
            
            # Rename columns to maturity labels
            rename_dict = {data.columns[i]: self.MATURITY_LABELS[region][i] for i in range(len(data.columns))}
            data = data.rename(columns=rename_dict)
            
            # Special case for AU: add 6m value
            if region == 'AU' and '6m' not in data.columns:
                data.insert(1, '6m', (data['3m'] + data['12m']) / 2)
            
            self.global_curves[region] = data
    
    def generate_global_comparison(self):
        """Generate chart comparing global yield curves"""
        self.fetch_global_curves()
        
        # Combine global curves
        global_df = pd.concat(self.global_curves.values(), axis=0, join='outer')
        global_transposed = global_df.transpose()
        df_global = global_transposed.reset_index().rename(columns={'index': 'Maturity'})
        df_global_long = df_global.melt(id_vars='Maturity', var_name='Curve', value_name='Yield')
        
        fig = self.create_line_chart(df_global_long, 'Maturity', 'Yield', 'Curve', 
                                    f'Global Yield Curves {date.today()}')
        # All curves with solid lines
        for trace in fig.data:
            trace.line.dash = 'solid'
        
        self.charts['Global_chart_html'] = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    def generate_aud_hedged_comparison(self):
        """Generate chart comparing AUD-hedged global yield curves"""
        # Retrieve FX data
        fx_data = {
            'AUDUSDspot': self.blp.bdh(tickers='AUDUSD Curncy', flds=['PX_LAST'], 
                              start_date='2023-04-01', end_date=self.today).tail(1),
            'AUDEURspot': self.blp.bdh(tickers='AUDEUR Curncy', flds=['PX_LAST'], 
                              start_date='2023-04-01', end_date=self.today).tail(1),
            'EURAUDspot': self.blp.bdh(tickers='EURAUD Curncy', flds=['PX_LAST'], 
                              start_date='2023-04-01', end_date=self.today).tail(1),
            'AUDUSDforward': self.blp.bdh(tickers='AUD1M BGN Curncy', flds=['PX_LAST'], 
                                 start_date='2023-04-01', end_date=self.today).tail(1),
            'EURAUDforward': self.blp.bdh(tickers='ADEU1M BGN Curncy', flds=['PX_LAST'], 
                                 start_date='2023-04-01', end_date=self.today).tail(1)
        }
        
        # Process FX forwards
        AUDUSDforward = (fx_data['AUDUSDforward'] * 12) / 10000
        EURAUDforward = (fx_data['EURAUDforward'] * 12) / 10000
        AUDEURimplied = 1 / (fx_data['EURAUDspot'].values[0] + EURAUDforward.values[0])
        AUDEURforward = (AUDEURimplied - fx_data['AUDEURspot'].values[0])
        
        # Make sure global curves are fetched
        if not self.global_curves:
            self.fetch_global_curves()
        
        # US adjusted for AUD
        us_hedged = {}
        for col in self.global_curves['US'].columns:
            val = (self.global_curves['US'][col] / 100) - AUDUSDforward.values[0] / (1 + (self.global_curves['US'][col] / 100)) * fx_data['AUDUSDspot'].values[0]
            us_hedged[col] = val
        self.hedged_curves['US'] = pd.DataFrame(us_hedged).mul(100).round(3)
        
        # EU adjusted for AUD
        eu_hedged = {}
        for col in self.global_curves['EU'].columns:
            val = (self.global_curves['EU'][col] / 100) - AUDEURforward / (1 + (self.global_curves['EU'][col] / 100)) * fx_data['AUDEURspot'].values[0]
            eu_hedged[col] = val
        self.hedged_curves['EU'] = pd.DataFrame(eu_hedged).mul(100).round(3)
        
        # AU remains the same
        self.hedged_curves['AU'] = self.global_curves['AU']
        
        # Combine hedged curves
        hedged_df = pd.concat(self.hedged_curves.values(), axis=0, join='outer')
        hedged_transposed = hedged_df.transpose()
        df_hedged = hedged_transposed.reset_index().rename(columns={'index': 'Maturity'})
        df_hedged_long = df_hedged.melt(id_vars='Maturity', var_name='Curve', value_name='Yield')
        
        fig = self.create_line_chart(df_hedged_long, 'Maturity', 'Yield', 'Curve', 
                                    f'Global Yield Curves (AUD Hedged) {date.today()}')
        for trace in fig.data:
            trace.line.dash = 'solid'
        
        self.charts['Globalhedged_chart_html'] = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
    
    def generate_all_charts(self):
        """Generate all charts in one call"""
        self.generate_individual_curves()
        self.generate_spread_charts()
        self.generate_global_comparison()
        self.generate_aud_hedged_comparison()
        return self.charts


# RUNNING THE YIELDCURVE VISUALISER
def run_yield_curve_analysis(blp):
    """Run the full yield curve analysis using the Bloomberg API"""
    visualizer = YieldCurveVisualizer(blp)
    charts = visualizer.generate_all_charts()
    
    # Access individual charts
    US_chart_html = charts['US_chart_html']
    EU_chart_html = charts['EU_chart_html']
    AU_chart_html = charts['AU_chart_html']
    Aus_comp_chart_html = charts['Aus_comp_chart_html']
    Aus_cred_chart_html = charts['Aus_cred_chart_html']
    Aus_FRN_chart_html = charts['Aus_FRN_chart_html']
    US_corp_chart_html = charts['US_corp_chart_html']
    US_cred_chart_html = charts['US_cred_chart_html']
    Global_chart_html = charts['Global_chart_html']
    Globalhedged_chart_html = charts['Globalhedged_chart_html']
    
    return charts

charts = run_yield_curve_analysis(blp)
US_chart_html = charts['US_chart_html']
EU_chart_html = charts['EU_chart_html']
AU_chart_html = charts['AU_chart_html']
Aus_comp_chart_html = charts['Aus_comp_chart_html']
Aus_cred_chart_html = charts['Aus_cred_chart_html']
Aus_FRN_chart_html = charts['Aus_FRN_chart_html']
US_corp_chart_html = charts['US_corp_chart_html']
US_cred_chart_html = charts['US_cred_chart_html']
Global_chart_html = charts['Global_chart_html']
Globalhedged_chart_html = charts['Globalhedged_chart_html']


########################
########################
########################
########################
# DAILY AND WEEKLY STUFF
########################

import pandas as pd
from datetime import datetime, timedelta
import plotly.offline as pyo

class DailyorWeeklyStuff:
    def __init__(self, blp):
        self.blp = blp

    # ---------- WEEKLY VALUATIONS ----------
    def get_weekly_valuations(self) -> dict:
        fields = ['PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO']
        fields_aus = ['PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'PX_TO_SALES_RATIO']

        sp500 = process_index(self.blp.bdh('SPX Index', fields, '1997-05-10', Per='W'))
        eur = process_index(self.blp.bdh('SXXP Index', fields, '1997-05-10', Per='W'))
        nky = process_index(self.blp.bdh('NKY Index', fields, '1997-05-10', Per='W'))
        asx = process_index(self.blp.bdh('AS51 Index', fields_aus, '1997-05-10', Per='W'))
        em_ = process_index(self.blp.bdh('MXEF Index', fields_aus, '1997-05-10', Per='W'))

        return {
            'eur': graph_performance(eur, 'Europe Valuation Composite - Weekly - STOXX 600'),
            'sp500': graph_performance(sp500, 'US Valuation Composite - Weekly - SP 500'),
            'asx': graph_performance(asx, 'AUS Valuation Composite - Weekly - ASX 200'),
            'nky': graph_performance(nky, 'Japan Valuation Composite - Weekly - Nikkei 225'), 'nky': graph_performance(em_, 'EM Valuation Composite - Weekly - MSCI EM'),
            'combined': graph_performance(
                pd.concat([nky, eur, sp500, asx, em_], axis=1, join='inner', keys=['Nikkei','Stoxx 600','SP500','ASX 200', 'MSCI EM']).xs('z_score', axis=1, level=1),
                'Valuation Composite - Weekly - Global'
            )
        }

    # ---------- EARNINGS REVISIONS ----------
    def get_earnings_revisions(self) -> str:
        one_year_ago = (datetime.today() - timedelta(days=252)).strftime("%Y-%m-%d")
        regions = ['MXWO Index','SPX Index','SPW Index', 'NKY Index','AS51 Index','MXEF Index','SXXP Index','MXKR Index','MXCN Index','UKX Index']
        names = ['World','US','US_Equal_Weight','Japan','Australia','Emerging Markets','Europe','Korea','China','UK']

        fwd = self.blp.bdh(regions, 'INDX_WEIGHTED_EST_ERN', one_year_ago, BEST_FPERIOD_OVERRIDE='BF').droplevel(1, axis=1)
        realized = self.blp.bdh(regions, 't12_eps_aggte', one_year_ago).droplevel(1, axis=1)

        fwd.columns = realized.columns = names
        revisions = (((fwd / realized) - 1) * 100).round(2)

        return graph_performance(revisions, 'Blended 4Q Forward Earnings Revisions (%)')

    # ---------- DAILY MARKET WATCH ----------
    def get_daily_market_watch(self, asset_path='temp_shortened.csv') -> tuple[dict, pd.DataFrame]:
        asset = pd.read_csv(asset_path, index_col=0).drop_duplicates()
        assets = asset.index.to_list()

        raw = self.blp.bdh(assets, 'tot_return_index_gross_dvds', '31-01-2010').droplevel(1, axis=1)
        raw.index = pd.to_datetime(raw.index)

        names = self.blp.bdp(assets, 'long_comp_name')
        types = self.blp.bdp(assets, 'SECURITY_TYP')

        def off_returns(series, days_ago):
            today = series.iloc[-2].name
            if today.weekday() == 0:
                ref = today - pd.DateOffset(days_ago + 3)
            else:
                ref = today - pd.DateOffset(days_ago + 1)

            prev_day = ref - pd.offsets.BusinessDay(1)
            next_day = ref + pd.offsets.BusinessDay(1)

            if days_ago > 252:
                return (series.loc[today] / series.loc[next_day])**(1/(days_ago/252)) - 1
            return (series.loc[today] / series.loc[next_day]) - 1

        ret_df = pd.DataFrame({k: off_returns(raw, v) for k, v in {
            '1d': 1, '3d': 3, '1w': 5, '1m': 21, '3m': 63,
            '6m': 126, '1y': 252, '3y': 756, '5y': 1260,
            '7y': 1764, '10y': 2520
        }.items()}).round(4).applymap(lambda x: f"{x*100:.2f}%")

        ret_df['Name'] = names
        ret_df['Asset Class'] = types
        df = ret_df.set_index('Name')

        equity = df[df['Asset Class'] == 'Equity Index'].drop('Asset Class', axis=1)
        debt = df[df['Asset Class'] == 'Fixed Income Index'].drop('Asset Class', axis=1)
        other = df[df['Asset Class'].isin(['Index', 'Commodity Index'])].drop('Asset Class', axis=1)

        tables = {
            'all': df.to_html(classes="table table-striped table-hover table-bordered", escape=False),
            'equity': equity.to_html(classes='table table-striped table-bordered', escape=False),
            'debt': debt.to_html(classes='table table-striped table-bordered', escape=False),
            'other': other.to_html(classes='table table-striped table-bordered', escape=False)
        }

        return tables, raw
    
    # ---------- CORRELATION ----------
    def get_stock_bond_corr(self, raw_df) -> str:
        corr = raw_df[['luattruu Index', 'spx index']].dropna()
        corr_pct = (corr / corr.shift()) - 1
        rolling = corr_pct['luattruu Index'].rolling(90).corr(corr_pct['spx index']).dropna()
        return simp_graph_performance(rolling, "90 Day Correlation (MSCI ACWI AND GLOBAL AGG)")

    # ---------- FACTOR PERFORMANCE ----------
    def get_factor_performance(self) -> dict:
        tickers = ['MXWO000V Index','MXWO000G Index','MXWOSC Index','MXWOLC Index','M1WOMOM Index','M1WOQU Index','M1WOMVOL Index','NU748615 Index','M1WOEV Index']
        names = ['Value','Growth','Small','Large','Momentum','Quality','Min Vol','Quality Smalls','Enhanced Value']

        df = self.blp.bdh(tickers, 'tot_return_index_gross_dvds', '31-07-2023', FX='USD').droplevel(1, axis=1)
        df.columns = names
        df.index = pd.to_datetime(df.index)

        returns = df.pct_change()
        ytd = returns[returns.index.year == 2025]
        cumulative = (1 + returns).cumprod()
        ytd_cumulative = (1 + ytd).cumprod()

        return {
            'full': graph_performance(cumulative.dropna(), "Global Factor Long only Performance"),
            'ytd': graph_performance(ytd_cumulative, "Global Factor Long only YTD Performance")
        }

    # ---------- SECTOR TABLES ----------
    def get_sector_performance_tables(self) -> dict:
        def load_sector_data(tickers):
            return self.blp.bdh(tickers, 'tot_return_index_gross_dvds', '31-07-2014', Per='M').droplevel(1, axis=1)

        sector_names = ['Mat', 'En', 'Fin', 'HC', 'CS', 'CD', 'IT', 'Real Est', 'Util', 'Comm', 'Indus']
        regions = {
            'us': ['s5matr Index', 's5enrs index', 's5finl index', 's5hlth index', 's5cond index', 's5cons index', 's5inft index', 's5rlst index', 's5util index', 's5tels index', 's5indu index'],
            'au': ['AS51MATL Index', 'as51engy index', 'as51fin index', 'as51hc index', 'as51cond index', 'as51cons index', 'as51it index', 'as51prop index', 'as51util index', 'as51tele index', 'as51indu index'],
            'jp': ['MXJP0MT Index', 'MXJP0EN Index', 'MXJP0FN Index', 'MXJP0HC Index', 'MXJP0CD Index', 'MXJP0CS Index', 'MXJP0IT Index', 'MXJP0RL Index', 'MXJP0UT Index', 'MXJP0TC Index', 'MXJP0IN Index'],
            'uk': ['MXGB0MT Index', 'MXGB0EN Index', 'MXGB0FN Index', 'MXGB0HC Index', 'MXGB0CD Index', 'MXGB0CS Index', 'MXGB0IT Index', 'MXGB0RL Index', 'MXGB0UT Index', 'MXGB0TC Index', 'MXGB0IN Index'],
            'eu': ['MXEU0MT Index', 'MXEU0EN Index', 'MXEU0FN Index', 'MXEU0HC Index', 'MXEU0CD Index', 'MXEU0CS Index', 'MXEU0IT Index', 'MXEU0RE Index', 'MXEU0UT Index', 'MXEU0TC Index', 'MXEU0IN Index'],
        }

        tables =  {
            region: calc_return_1m_to_10y(load_sector_data(tickers), sector_names).to_html(escape=False, classes='table table-striped table-bordered')
            for region, tickers in regions.items()
        }

        return tables
    

    def get_all_asset_class_z_scores(self) -> str:
        import pandas as pd
        from datetime import datetime
        import plotly.graph_objects as go
        from plotly.offline import plot
        from scipy.stats.mstats import winsorize

        # Set dates dynamically
        today = pd.Timestamp.today()
        prev_month = today - pd.DateOffset(months=1)
        start_date = (today - pd.DateOffset(years=25)).strftime('%Y-%m-%d')
        end_date = today.strftime('%Y-%m-%d')

        # Create formatted legend labels based on dates
        current_label = today.strftime('%b %Y')
        prev_label = prev_month.strftime('%b %Y')

        invert_fields = ['yield_to_worst', 'OAS_SPREAD_MID', 'px_last']

        groups = {
            "Bonds": [
                {"name": "US Treasuries", "ticker": "LGTRTRUU Index", "field": "yield_to_worst"}, 
                {"name": "US HY OAS", "ticker": "LF98OAS Index", "field": "px_last"}, 
                {"name": "US Inv Grade Credit OAS", "ticker": "LUCROAS Index", "field": "px_last"},
                {"name": "AUS Corp Bond Comp", "ticker": "BACR0 Index", "field": "OAS_SPREAD_MID"},
                {"name": "Ausbond Credit FRN", "ticker": "BAFRN0 Index", "field": "OAS_SPREAD_MID"},
                {"name": "Euro IG OAS", "ticker": "LECPOAS Index", "field": "px_last"}
            ],
            "Equities": [
                {"name": "MSCI World", "ticker": "MXWO Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
                {"name": "ASX 200", "ticker": "AS51 Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
                {"name": "MSCI Europe", "ticker": "MXEU Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
                {"name": "MSCI EM", "ticker": "MXEF Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
                {"name": "MSCI World Small Cap", "ticker": "MXWOSC Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
            ],
            "Real Assets": [
                {"name": "ASX 200 A-Reit", "ticker": "AS51PROP Index", "field": "px_to_tang_bv_per_sh"},
                {"name": "FTSE EORA/NAREIT Dev", "ticker": "ENGL Index", "field": "px_to_tang_bv_per_sh"},
                {"name": "S&P Global Infra", "ticker": "SPGTIND Index", "field": "px_to_tang_bv_per_sh"},
            ]
        }

        # Flatten asset list
        assets = []
        for group, lst in groups.items():
            for asset in lst:
                asset['group'] = group
                assets.append(asset)

        def get_zscore(df, target_date):
            if df.empty: return None, None
            values_winsor = winsorize(df['value'], limits=[0.01, 0.01])
            mean_val = values_winsor.mean()
            std_val = values_winsor.std()
            df['z_score'] = (values_winsor - mean_val) / std_val
            try:
                idx = df.index.get_loc(target_date, method='nearest')
                return df.iloc[idx]['z_score'], df.index[0].strftime('%b %d, %Y')
            except:
                return None, df.index[0].strftime('%b %d, %Y')

        results = []
        for asset in assets:
            df = self.blp.bdh(asset['ticker'], flds=asset['field'], start_date=start_date, end_date=end_date)
            df.columns = ['value']
            df.dropna(inplace=True)

            z_current, first_valid = get_zscore(df.copy(), today)
            z_prev, _ = get_zscore(df.copy(), prev_month)

            if asset['field'] in invert_fields:
                if z_current is not None: z_current *= -1
                if z_prev is not None: z_prev *= -1
                label_field = f"Inverted {asset['field']}"
            else:
                label_field = asset['field']

            results.append({
                "name": asset['name'],
                "field": label_field,
                "group": asset['group'],
                "z_current": z_current,
                "z_prev": z_prev,
                "first_valid": first_valid
            })

        zscore_df = pd.DataFrame(results)
        fig = go.Figure()

        for i, row in zscore_df.iterrows():
            x0, x1 = i - 0.4, i + 0.4
            for y0, y1, color in [(2, 3, "gainsboro"), (-3, -2, "gainsboro"), (1, 2, "lightgray"), (-2, -1, "lightgray"), (-1, 1, "gray")]:
                fig.add_shape(type="rect", x0=x0, x1=x1, y0=y0, y1=y1, fillcolor=color, opacity=0.2 if 'gain' in color else 0.3, layer="below", line_width=0)

        for i, row in zscore_df.iterrows():
            if row['z_current'] is not None:
                fig.add_trace(go.Scatter(
                    x=[row['name']], y=[row['z_current']],
                    mode='markers+text',
                    name=current_label if i == 0 else None,
                    marker=dict(symbol='circle', size=14, color="#30415f"),
                    text=[f"{row['z_current']:.2f}"],
                    textposition='top center',
                    showlegend=i == 0,
                    textfont=dict(family="Montserrat")
                ))

            if row['z_prev'] is not None:
                fig.add_trace(go.Scatter(
                    x=[row['name']], y=[row['z_prev']],
                    mode='markers',
                    name=prev_label if i == 0 else None,
                    marker=dict(symbol='triangle-down', size=14, color="#a8c686"),
                    showlegend=i == 0
                ))

        fig.update_layout(
            title={'text': 'Asset Class Valuations (Z-scores based on 25-year average valuation measures)', 'font': {'family': 'Montserrat', 'size': 18}},
            yaxis=dict(title='Z-score', range=[-3.5, 3], zeroline=True, zerolinewidth=2, zerolinecolor='black', showgrid=False, tickfont=dict(family="Montserrat")),
            xaxis=dict(
                tickvals=zscore_df['name'].tolist(),
                ticktext=[
                    f"{row['name']}<br><span style='font-size:11px;color:gray'>{row['field']}<br>{row['first_valid']}</span>"
                    for _, row in zscore_df.iterrows()
                ],
                showgrid=False,
                tickfont=dict(family="Montserrat")
            ),
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(family="Montserrat")),
            height=720,
            width=max(900, 150 + 100 * len(zscore_df)),
            margin=dict(t=80, b=160),
            font=dict(family="Montserrat")
        )

        return plot(fig, output_type='div', include_plotlyjs='cdn')


model = DailyorWeeklyStuff(blp)

dmw_tables, raw = model.get_daily_market_watch()
equity_styled = dmw_tables['equity']
debt_styled = dmw_tables['debt']
other_styled = dmw_tables['other']
valuation_charts = model.get_weekly_valuations()
graph_for_regional_equity_ytd = valuation_charts['combined']
factor_charts = model.get_factor_performance()
graph_for_factor_equity_ytd = factor_charts['ytd']
graph_for_factor_equity = factor_charts['full']
earnings_rev_html = model.get_earnings_revisions()
corr_chart = model.get_stock_bond_corr(raw)
sector_tables = model.get_sector_performance_tables()
aashna_all_asset_class_z_score_valuations_html = model.get_all_asset_class_z_scores()

########################
########################
########################
########################
# VALUATION , MONTHLY STUFF
########################

class ValuationAnalytics:
    def __init__(self, blp):
        self.blp = blp

        # Core configs
        self.frequency = 'M'
        self.start_date = '1997-05-10'
        self.lower = 0.01
        self.upper = 0.99

        # Valuation metrics and labels
        self.valuation_metric_list = ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO']
        self.valuation_list = ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']

        # Regional market tickers and names
        self.regional_list = ['MXWO Index','SPX Index','SPW Index', 'NKY Index','AS51 Index','AS38 Index','MXEF Index','SXXP Index','MXKR Index','MXCN Index','UKX Index', 'MXWOU Index']
        self.regionals_name = ['World','US','US Equal Weight', 'Japan','Australia','Aussie Smalls', 'EM','Europe','Korea','China','UK', 'World ex US']

        # Factor index tickers and names
        self.factors_list = ['MXWO000V Index', 'M1WOEV Index', 'MXWO000G Index', 'M1WOQU Index','MXWOSC Index','MXWOLC Index','M1WOMVOL Index']
        self.factors_names = ['Value', 'Enhanced Value', 'Growth', 'Quality', 'Small', 'Large', 'Low Volatility']

    def get_monthly_valuation_data(self):
        raw = self.blp.bdh(tickers=self.regional_list, flds=self.valuation_metric_list, start_date=self.start_date, Per=self.frequency)
        return clean_data_after_bloomberg(raw, self.regionals_name, self.valuation_list)

    def get_factor_valuation_data(self):
        raw = self.blp.bdh(tickers=self.factors_list, flds=self.valuation_metric_list, start_date=self.start_date, Per=self.frequency)
        return clean_data_after_bloomberg(raw, self.factors_names, self.valuation_list)

    def get_region_factor_graphs(self, regional_data, factor_data):
        generate_graphs(regional_data, self.regionals_name, "reig_graphs")
        generate_graphs(factor_data, self.factors_names, "factor_graphs")

        output = {
            'regional': {
                metric: "".join(globals()[f"reig_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'EV_Trailing_EBITDA', 'Price_to_Sales', 'Valuation_Composite']
            },
            'factor': {
                metric: "".join(globals()[f"factor_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'EV_Trailing_EBITDA', 'Price_to_Sales', 'Valuation_Composite']
            }
        }
        return output

    def cross_sectional_time_series(self, reg_data, fac_data):
        long = ['Value', 'Small', 'EM', 'World ex US', 'US Equal Weight']
        short = ['Growth', 'Large', 'World', 'US', 'US']
        metric = 'Valuation Composite'

        xs_df = pd.DataFrame({
            'Global Value / Growth': fac_data[0][long[0]][metric] - fac_data[0][short[0]][metric],
            'Small / Large': fac_data[0][long[1]][metric] - fac_data[0][short[1]][metric],
            'EM / DM': reg_data[0][long[2]][metric] - reg_data[0][short[2]][metric],
            'Intl ex US / US': reg_data[0][long[3]][metric] - reg_data[0][short[3]][metric],
            'S&P Equal / S&P 500': reg_data[0][long[4]][metric] - reg_data[0][short[4]][metric]
        }, index=fac_data[0].index)

        return graph_performance(xs_df, 'Cross-Sectional Valuation Composite Spreads')

    def cross_sectional_current_tables(self, reg_data, fac_data):
        return {
            'factor_table': cross_sectional_current_table_maker(fac_data),
            'regional_table': cross_sectional_current_table_maker(reg_data)
        }

    def get_aussie_valuations(self):
        tickers = ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS51 Index']
        labels = ['Forward PE', 'Price to Book', 'CAPE', 'Price to Sales']
        fields = ['BEST_PE_RATIO', 'PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO', 'INDX_PX_SALES']
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        raw = self.blp.bdh(tickers=tickers, flds=fields, start_date='2003-05-10', Per='M')
        return clean_data_after_bloomberg(raw, names, labels)

    def get_aussie_graphs(self, data):
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        generate_graphs(data, names, "aus_reig_graphs")
        return {
            metric: "".join(globals()[f"aus_reig_graphs_{metric.replace(' ', '_')}"]) for metric in ['Forward_PE', 'Price_to_Book', 'CAPE', 'Price_to_Sales', 'Valuation_Composite']
        }

    def get_aussie_earnings_revision(self, one_year_ago):
        tickers = ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS52 Index']
        names = ['Banks','Resources','Equal Weight','Materials','ASX 200']
        fwd = self.blp.bdh(tickers=tickers, flds=['INDX_WEIGHTED_EST_ERN'], start_date=one_year_ago, BEST_FPERIOD_OVERRIDE='BF').droplevel(1, axis=1)
        realized = self.blp.bdh(tickers=tickers, flds=['t12_eps_aggte'], start_date=one_year_ago).droplevel(1, axis=1)
        fwd.columns = names
        realized.columns = names

        rev_table = (((fwd / realized) - 1) * 100).round(2)
        return graph_performance(rev_table, 'Blended 4Q Forward Earnings Revisions (%)')

va = ValuationAnalytics(blp)

# Auto-uses preloaded tickers and labels
regional_data = va.get_monthly_valuation_data()
factor_data = va.get_factor_valuation_data()
graphs = va.get_region_factor_graphs(regional_data, factor_data)
xs_graph = va.cross_sectional_time_series(regional_data, factor_data)
aussie_data = va.get_aussie_valuations()
aus_graphs = va.get_aussie_graphs(aussie_data)
# Get one year ago from today
one_year_ago = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
aussie_blend4q_earnings_rev = va.get_aussie_earnings_revision(one_year_ago)
blend4q_earnings_rev = model.get_earnings_revisions()
tables = va.cross_sectional_current_tables(regional_data, factor_data)
regional_val_table = tables['regional_table']
factor_val_table = tables['factor_table']


########################
########################
########################
# MACRO
########################

import pandas as pd
import plotly.express as px
from plotly.offline import plot

class MacroAnalytics:
    def __init__(self, blp):
        self.blp = blp

    def get_economic_surprise_index(self):
        tickers = ['CESIUSD Index', 'CESIEUR Index', 'CESIAUD Index', 'CESICNY Index']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='2021-11-01', Per='M').droplevel(1, axis=1)
        df.columns = ['US', 'EU', 'AU', 'China']
        return graph_performance(df, "Citi Economic Surprise Indices")

    def get_weekly_economic_surprise(self):
        tickers = ['GTII10 Govt', 'CESIEUR Index', 'CESIAUD Index', 'GTAUDII10Y Govt']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='2021-11-01', Per='W').droplevel(1, axis=1)
        df.columns = ['US', 'EU', 'AU', 'China']
        return graph_performance(df, "Citi Economic Surprise Indices")

    def get_gdp_forecast_2025(self):
        tickers = ['ECGDUS 25 Index', 'ECGDAU 25 Index', 'ECGDGB 25 Index', 'ECGDEU 25 Index', 'ECGDCN 25 Index',
                   'ECGDJP 25 Index', 'ECGDKR 25 Index', 'ECGDBR 25 Index', 'ECGDIN 25 Index']
        names = ['USA', 'AUS', 'GBR', 'EUR', 'CHN', 'JAP', 'KOR', 'BRA', 'IND']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='2023-09-01', Per='W').droplevel(1, axis=1)
        df.columns = names
        return graph_performance(df, "Bloomberg Consensus Real GDP Forecast YoY 2025")

    def get_manufacturing_pmi_chart(self):
        tickers = ['NAPMPMI Index','MPMIAUMA Index','MPMIGBMA Index','MPMIEZMA Index','CPMINDX Index',
                   'MPMIJPMA Index','MPMIKRMA Index','MPMIBRMA Index','MPMIINMA Index']
        names = ['USA','AUS','GBR','EUR','CHN','JAP','KOR','BRA','IND']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='2020-10-10', Per='M').droplevel(1, axis=1).dropna()
        df.index = pd.to_datetime(df.index)
        df.columns = names
        graph_df = pd.concat([df.iloc[-1], df.iloc[-12]], axis=1)
        graph_df.columns = ['Current', '1 Year Ago']

        fig = px.bar(graph_df, x=graph_df.index, y=['Current', '1 Year Ago'],
                     title="Manufacturing PMI: Current vs 1 Year Ago",
                     labels={"value": "PMI Index", "index": ""},
                     barmode='group',
                     color_discrete_sequence=['#30415f', '#669bbc'])

        fig.update_layout(
            font_family="Montserrat",
            title={"font": {"size": 22}},
            yaxis=dict(title="PMI Index", titlefont=dict(color="black"), tickfont=dict(color="black"), gridcolor="lightgray"),
            plot_bgcolor="white",
            paper_bgcolor="white",
            width=950,
            height=600,
            legend=dict(orientation="h", y=-0.075, x=0.5, xanchor="center")
        )
        return plot(fig, output_type='div', include_plotlyjs='cdn')

    def get_leading_indicator_zscore_chart(self):
        df = self.blp.bdh(tickers='LEI YOY Index', flds=['px_last'], start_date='1960-01-01', Per='M')
        df_z = (df - df.median()) / df.std()
        return simp_graph_performance(df_z.droplevel(1, axis=1), "US Leading Economic Indicator YoY")

    def get_real_gdp_table(self):
        tickers = ['GDP CYOY Index', 'AUNAGDPY Index', 'UKGRABIY Index', 'EUGNEMUY Index',
                   'CNGDPYOY Index','JGDPNSAQ Index','KOGDPYOY Index','BZGDYOY% Index','IGQREGDY Index']
        names = ['USA','AUS','GBR','EUR','CHN','JAP','KOR','BRA','IND']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='2020-10-10', Per='Q').droplevel(1, axis=1).dropna()
        df.index = pd.to_datetime(df.index)
        latest = pd.concat([df.iloc[-1], df.iloc[-5]], axis=1)
        latest.index = names
        latest.columns = ['Current', '1 Year Ago']

        return (
            latest.style
            .format(precision=2)
            .set_table_styles([
                {'selector': 'td:hover', 'props': [('background-color', '#30415f')]},
                {'selector': 'th:not(.index_name)', 'props': [('background-color', '#30415f'), ('color', 'white'), ('text-align', 'center')]},
                {'selector': 'td', 'props': [('font-size', '14px'), ('text-align', 'center'), ('width', '80px'), ('border', '1px solid #ddd')]},
                {'selector': 'th', 'props': [('text-align', 'left'), ('border', '1px solid #ddd'), ('width', '80px')]},  # <-- fixed here
                {'selector': 'table', 'props': [('border-collapse', 'collapse')]}
            ])
            .to_html(table_attrs={'id': 'marketWatchTable', 'class': 'display'})
        )


    def get_recession_probability_zscores(self):
        tickers = ['LEI YOY Index','NFCIINDX Index','OEUSZREX Index','USGG2YR Index','FEDL01 Index','NAPMNEWO Index','NHSPSTOT Index']
        df = self.blp.bdh(tickers=tickers, flds=['px_last'], start_date='1960-01-01', Per='M').droplevel(1, axis=1)
        return (df - df.mean()) / df.std()

macro = MacroAnalytics(blp)

eco_surpris_df_html = macro.get_economic_surprise_index()
gdp_consensus_html = macro.get_gdp_forecast_2025()
eco_surpris_weekly_html = macro.get_weekly_economic_surprise()
manufac = macro.get_manufacturing_pmi_chart()
lei_z_table = macro.get_leading_indicator_zscore_chart()
real_gdp_table = macro.get_real_gdp_table()
recession_probably_z_scores = macro.get_recession_probability_zscores()


########################
########################
########################
# TECHNICALS
########################

import pandas as pd
import plotly.graph_objects as go

class TechnicalAnalytics:
    def __init__(self, blp):
        self.blp = blp
        self.start_date = '2005-06-01'
        self.tickers = ['AS51 INDEX', 'spx index', 'nky index', 'mxeu index', 'mxef index',
                        'UKX index', 'shcomp index', 'KOSPI Index', 'NDX Index', 'spw index', 'mxwd index']
        self.names = ['ASX200', 'S&P 500', 'Nikkei 225', 'MSCI Europe', 'MSCI EM',
                      'FTSE 100', 'Shanghai Composite', 'KOSPI Index', 'Nasdaq Comp', 'S&P 500 Equal Weight', 'MSCI ACWI']

    def get_major_index_data(self):
        df = self.blp.bdh(tickers=self.tickers, flds=['px_last'], start_date=self.start_date).droplevel(axis=1, level=1)
        df.columns = self.names
        return df.ffill()

    def plot_moving_averages(self, dataframe):
        graph_html = {}
        ma_specs = {
            '50d_MA': {'color': 'silver', 'dash': 'dot'},
            '200d_MA': {'color': 'steelblue', 'dash': 'dot'}
        }
        for index_name in dataframe.columns:
            temp_df = dataframe[[index_name]].copy()
            temp_df['50d_MA'] = temp_df[index_name].rolling(window=50).mean()
            temp_df['200d_MA'] = temp_df[index_name].rolling(window=200).mean()

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=temp_df.index,
                y=temp_df[index_name],
                mode='lines',
                name=f'{index_name} Price',
                line=dict(color='#30415f', width=2)
            ))
            for ma_type, style in ma_specs.items():
                fig.add_trace(go.Scatter(
                    x=temp_df.index,
                    y=temp_df[ma_type],
                    mode='lines',
                    name=ma_type,
                    line=dict(color=style['color'], dash=style['dash'], width=2)
                ))

            fig.update_layout(
                title=f'{index_name} Moving Averages',
                xaxis_title='Date',
                yaxis_title='Price',
                template='plotly_white',
                hovermode='x unified',
                font=dict(family="Montserrat, sans-serif"),
                title_font=dict(family="Montserrat, sans-serif"),
                legend_font=dict(family="Montserrat, sans-serif"),
                width=1400,
                height=700
            )
            fig.update_xaxes(title_font=dict(family="Montserrat, sans-serif"))
            fig.update_yaxes(title_font=dict(family="Montserrat, sans-serif"))

            graph_html[index_name] = fig.to_html(full_html=False, include_plotlyjs='cdn')

        return graph_html

ta = TechnicalAnalytics(blp)

index_data = ta.get_major_index_data()
technicals_graphs_html = ta.plot_moving_averages(index_data)

########################
########################
########################

In [ ]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Innova Asset Mangagement Internal Dashboard</title>
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700&display=swap" rel="stylesheet">
  <!-- Bootstrap CSS -->
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    /* Base styles */
    body {{
      margin: 0;
      font-family: 'Montserrat', sans-serif;
    }}
    /* Header Section */
    .site-header {{
      width: 100%;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}
    .palette {{
      display: flex;
    }}
    .palette .color-block {{
      flex: 1;
      height: 8px;
    }}
    .header-title-container {{
      background-color: #30415f;
      padding: 10px;
      text-align: center;
      color: #fff;
      position: relative;
    }}
    /* Home button styling */
    .home-button {{
      position: absolute;
      right: 20px;
      top: 10px;
    }}
    /* Main container for sidebar + content */
    .main-container {{
      display: flex;
      height: calc(100vh - 60px); /* Adjust based on header height */
    }}
    .sidebar {{
      width: 250px;
      background-color: #f5f5f5;
      padding: 10px;
      border-right: 1px solid #ddd;
      overflow-y: auto;
    }}
    .content {{
      flex-grow: 1;
      padding: 20px;
      overflow-y: auto;
    }}
    /* First-level tabs styling for sidebar (unchanged) */
    #sidebar-menu > a.list-group-item {{
      background-color: #30415f;
      color: #fff;
      font-weight: bold;
      border: none;
    }}
    #sidebar-menu > a.list-group-item:hover {{
      background-color: #30415f; 
    }}
    /* Sub-tabs remain default */
    .list-group-item {{
      cursor: pointer;
    }}
    /* Image styling */

    .content-pane div.js-plotly-plot {{
        min-height: 400px;
      }}
      
    .content-pane img {{
      display: block;
      margin: 20px auto; /* Adds spacing between images */
      max-width: 100%; /* Ensures images don’t overflow */
    }}

    /* Graph styling: center divs, canvases, iframes, and svgs inside .content-pane */
    .content-pane div,
    .content-pane canvas,
    .content-pane iframe,
    .content-pane svg {{
      display: block;
      margin: 20px auto; /* Centers the element */
      max-width: 100%;   /* Prevents overflow */
    }}

    /* Horizontal navigation tabs styling */
    .main-nav {{
      margin-bottom: 0;
    }}
    .nav-tabs .nav-link {{
      color: #30415f;  /* Changed text color */
    }}


    /* Dropdown styling */
    .graph-selector {{
      display: block;
      width: 100%;
      max-width: 500px;
      margin: 20px auto;
      padding: 8px 12px;
      font-size: 16px;
      border: 1px solid #ccc;
      border-radius: 4px;
      background-color: white;
      font-family: 'Montserrat', sans-serif;
    }}
    
    /* Graph container */
    .graph-container {{
      margin-top: 30px;
    }}

  
    .sector-table {{
      margin: 0 auto;
      text-align: center;
    }}
    .sector-table th, .sector-table td {{
      text-align: center;
      padding: 8px;
    }}
    </style>

  </style>
</head>
<body>
  <!-- Header Section with Palette and Title -->
  <header class="site-header">
    <div class="palette">
      <div class="color-block" style="background-color: #30415f;"></div>
      <div class="color-block" style="background-color: #30415f;"></div>
    </div>
    <div class="header-title-container">
      <h1>Innova Asset Mangagement Internal Dashboard</h1>
      <a href="#" class="btn btn-light home-button" onclick="showContent('landing')">Home</a>
    </div>
  </header>

  <!-- Main Tabs Horizontal Navigation -->
  <nav class="main-nav">
    <ul class="nav nav-tabs">
      <li class="nav-item">
        <a class="nav-link" data-main-tab="innova" href="#" onclick="showMainTab('innova')">Innova SMA Analytics</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="dmw" href="#" onclick="showMainTab('dmw')">Daily Indicators</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vf" href="#" onclick="showMainTab('vf')">Equity Valuations</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vvf" href="#" onclick="showMainTab('vvf')">Earnings & Fundamentals</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="macro" href="#" onclick="showMainTab('macro')">Macroeconomic</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="tech" href="#" onclick="showMainTab('tech')">Technicals / Sentiment / Risk Aversion</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="misc" href="#" onclick="showMainTab('misc')">Bond Futures and Yield Curve</a>
      </li>
    </ul>
  </nav>
  
  <!-- Main Container: Sidebar (Subtabs) + Content Area -->
  <div class="main-container">
    <!-- Sidebar for Subtabs -->
    <div class="sidebar" id="subtab-sidebar">
      <!-- Innova SMA Analytics Subtabs -->
      <div class="subtabs" id="subtabs-innova" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmww1')">Traffic Lights</a>
          <a class="list-group-item" onclick="showContent('dmww3')">Target Spreadsheets</a>
          <a class="list-group-item" onclick="showContent('dmww4')">Innova Charts 2025</a>
        </div>
      </div>
      <!-- Daily Market Watch Subtabs -->
      <div class="subtabs" id="subtabs-dmw" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmw1')">Asset Performance</a>
          <a class="list-group-item" onclick="showContent('dmw2')">Reigonal Graph Perf</a>
          <a class="list-group-item" onclick="showContent('dmw3')">Factor Graph Perf</a>
          <a class="list-group-item" onclick="showContent('dmw4')">Sector Performance</a>
          <a class="list-group-item" onclick="showContent('dmw5')">Daily Global Valuations</a>
        </div>
      </div>
      <!-- Equity Valuations Subtabs -->
      <div class="subtabs" id="subtabs-vf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vf8')">Dedicated Aussie Valuation</a>
          <a class="list-group-item" onclick="showContent('vf0')">Reigonal Time-Series Val</a>
          <a class="list-group-item" onclick="showContent('vf6')">Cross-Sectional Valuation</a>
          <a class="list-group-item" onclick="showContent('vf7')">Factor Valuations</a>
        </div>
      </div>
      <!-- Earnings & Fundamentals Subtabs -->
      <div class="subtabs" id="subtabs-vvf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vvf1')">Regional Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf9')">Magnificent 7 Earnings & Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf2')">Factor Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf3')">Sources of Return & Box Valuations</a>
          <a class="list-group-item" onclick="showContent('vvf4')">Key Company Earnings</a>
          <a class="list-group-item" onclick="showContent('vvf5')">S&P Profit Margins</a>
          <a class="list-group-item" onclick="showContent('vvf6')">Global Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf7')">Australia Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf8')">Analyst Estimate... list?</a>
        </div>
      </div>
      <!-- Macroeconomic Subtabs -->
      <div class="subtabs" id="subtabs-macro" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('macro1')">Macroeconomic Regime</a>
          <a class="list-group-item" onclick="showContent('macro2')">US Growth & Inflation</a>
          <a class="list-group-item" onclick="showContent('macro3')">Global Eco Surprise</a>
          <a class="list-group-item" onclick="showContent('macro5')">US Recession Probability</a>
          <a class="list-group-item" onclick="showContent('macro6')">US Stock Bond Correlation</a>
        </div>
      </div>
      <!-- Technicals / Sentiment / Risk Aversion Subtabs -->
      <div class="subtabs" id="subtabs-tech" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('tech1')">Moving Day Average Graphs</a>
          <a class="list-group-item" onclick="showContent('tech2')">Table of Index Technicals</a>
          <a class="list-group-item" onclick="showContent('tech3')">Volatility of Asset Classes</a>
          <a class="list-group-item" onclick="showContent('tech4')">Net Flows</a>
          <a class="list-group-item" onclick="showContent('tech5')">Fear / Greed Type Indicators</a>
        </div>
      </div>
      <!-- Bond Futures and Yield Curve Subtabs -->
      <div class="subtabs" id="subtabs-misc" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('misc1')">Rate Cut/Hike Pricing</a>
          <a class="list-group-item" onclick="showContent('misc2')">Yield Curves</a>
          <a class="list-group-item" onclick="showContent('misc3')">AU Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc4')">Global Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc5')">TIPS</a>
        </div>
      </div>
    </div>
    
    <!-- Content Area for Charts -->
    <div class="content" id="content-area">
      <!-- Landing Page -->
      <div id="landing" class="content-pane">
        <h2>Welcome Innova Team</h2>
        <p>Please use the tabs above to navigate the different sections. Below are some relevant weekly updates as of {today_date} in macro/markets:</p>
        <ul>
          <li><a href="https://www.cnbc.com/finance/" target="_blank">CNBC Finance Top Headlines for {today_date}</a></li>
          <li> <a href="https://tradingeconomics.com/calendar" target="_blank">Economic Calender/Releases as of {today_date}</a></li>
          <br>
          <br>
          <h2>Top News of the Week:</h2>
          <li> JPMorgan noted that the tariffs would hike taxes on Americans by $660 billion a year, the largest tax increase in recent memory by a longshot. It will cause prices to surge, too, adding 2% to the Consumer Price Index, a measure of US inflation that has struggled to come back down to earth in recent years.</li>
          <li></li>
          <li><a href="https://www.atlantafed.org/cqer/research/gdpnow" target="_blank">Latest GDP Nowcast is {next_24_chars}</a></li>
          <div>{tariff_2025}</div>
          <div>{tariff_2025_asia}</div>
        </ul>
      </div>
      
      <!-- Innova SMA Analytics Content Panes -->
      <div id="dmww1" class="content-pane" style="display:none;">
        <div>{aashna_all_asset_class_z_score_valuations_html}</div>
      </div>
      <div id="dmww3" class="content-pane" style="display:none;">
        <div>{Funda}</div>
        <div>{Fla}</div>
        <div>{Trad}</div>
        <div>{Cfs_fc}</div>
      </div>
      <div id="dmww4" class="content-pane" style="display:none;">
        <div>{basecase}</div>
        <div>{eco_surprise}</div>
        <div>{concentration}</div>
        <div>{cape_chart}</div>
        <div>{returns_2022}</div>
        <div>{crsp_dimensional}</div>
        <div>{region_positioning}</div>
      </div>

      <!-- Daily Market Watch Content Panes -->
      <div id="dmw1" class="content-pane" style="display:none;">
       <h2 style="text-align: center;">'Equities only'</h2>
       <div style="display: flex; justify-content: center; margin-top: 20px;">
        {equity_styled}
        </div>
        <h2>{'Fixed Income only'}</h2>
        <div>{debt_styled}</div>
        <h2>{'Other'}</h2>
        <div>{other_styled}</div>
      </div>
      <div id="dmw2" class="content-pane" style="display:none;">
        {graph_for_regional_equity_ytd}
      </div>
      <div id="dmw3" class="content-pane" style="display:none;">
        <div>{graph_for_factor_equity_ytd}</div>
        <div class="plot-container">{graph_for_factor_equity}</div>
      </div>
      <div id="dmw4" class="content-pane" style="display: none; overflow-x: auto;">
        <h2>US Sector</h2>
        <div>{sector_tables['us']}</div>
        <h2>AU Sector</h2>
        <div>{sector_tables['au']}</div>
        <h2>Japan Sector</h2>
        <div>{sector_tables['jp']}</div>
        <h2>EU Sector</h2>
        <div>{sector_tables['eu']}</div>
        <h2>UK Sector</h2>
        <div>{sector_tables['uk']}</div>
      </div>


      <div id="dmw5" class="content-pane" style="display:none;">
        <div>{valuation_charts['combined']}</div>
        <div>{valuation_charts['sp500']}</div>
        <div>{valuation_charts['eur']}</div>
        <div>{valuation_charts['asx']}</div>
        <div>{valuation_charts['nky']}</div>
      </div>

      
      <!-- Equity Valuations Content Panes -->
      <div id="vf8" class="content-pane" style="display:none;">
        <div>{aussie_blend4q_earnings_rev}</div>

        <select id="ausValueSelector" class="graph-selector" onchange="showAusValuation(this.value)">
        <option value="">-- Select a valuation metric --</option>
        <option value="aus_forward_pe">Forward PE</option>
        <option value="aus_cape">CAPE Ratio</option>
        <option value="aus_price_to_book">Price to Book</option>
        <option value="aus_price_to_sales">Price to Sales</option>
        <option value="aus_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
        <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
        </div>
        <div id="aus_forward_pe" style="display: none;">{aus_graphs['Forward_PE']}</div>
        <div id="aus_cape" style="display: none;">{aus_graphs['CAPE']}</div>
        <div id="aus_price_to_book" style="display: none;">{aus_graphs['Price_to_Book']}</div>
        <div id="aus_price_to_sales" style="display: none;">{aus_graphs['Price_to_Sales']}</div>
        <div id="aus_valuation_composite" style="display: none;">{aus_graphs['Valuation_Composite']}</div>
        </div>

      </div>

      <div id="vf0" class="content-pane" style="display:none;">
      <div> {regional_val_table} </div>

        <select id="reigValueSelector" class="graph-selector" onchange="reigAusValuation(this.value)">
          <option value="">-- Select a valuation metric --</option>
          <option value="_forward_pe">Forward PE</option>
          <option value="_cape">CAPE Ratio</option>
          <option value="_price_to_book">Price to Book</option>
          <option value="_price_to_sales">Price to Sales</option>
          <option value="_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
          <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
          </div>
          <div id="_forward_pe" style="display: none;">{graphs['regional']['Forward_PE']}</div>
          <div id="_cape" style="display: none;">{graphs['regional']['CAPE']}</div>
          <div id="_price_to_book" style="display: none;">{graphs['regional']['Price_to_Book']}</div>
          <div id="_price_to_sales" style="display: none;">{graphs['regional']['Price_to_Sales']}</div>
          <div id="_valuation_composite" style="display: none;">{graphs['regional']['Valuation_Composite']}</div>

        </div>
      </div>

      <div id="vf6" class="content-pane" style="display:none;">
        {xs_graph}
      </div>
      <div id="vf7" class="content-pane" style="display:none;">
        <div> {factor_val_table} </div>
        <select id="ValueSelector" class="graph-selector" onchange="showValuation(this.value)">
          <option value="">-- Select a valuation metric --</option>
          <option value="factor_forward_pe">Forward PE</option>
          <option value="factor_cape">CAPE Ratio</option>
          <option value="factor_price_to_book">Price to Book</option>
          <option value="factor_price_to_sales">Price to Sales</option>
          <option value="factor_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
          <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
          </div>
          <div id="factor_forward_pe" style="display: none;">{graphs['factor']['Forward_PE']}</div>
          <div id="factor_cape" style="display: none;">{graphs['factor']['CAPE']}</div>
          <div id="factor_price_to_book" style="display: none;">{graphs['factor']['Price_to_Book']}</div>
          <div id="factor_price_to_sales" style="display: none;">{graphs['factor']['Price_to_Sales']}</div>
          <div id="factor_valuation_composite" style="display: none;">{graphs['factor']['Valuation_Composite']}</div>

        </div>
      </div>

      <!-- Earnings & Fundamentals Content Panes -->
      <div id="vvf1" class="content-pane" style="display:none;">
        {blend4q_earnings_rev}
      </div>
      <div id="vvf9" class="content-pane" style="display:none;">
      </div>
      <div id="vvf2" class="content-pane" style="display:none;">
      </div>
      <div id="vvf3" class="content-pane" style="display:none;">
      </div>
      <div id="vvf4" class="content-pane" style="display:none;">
      </div>
      <div id="vvf5" class="content-pane" style="display:none;">
      </div>
      <div id="vvf6" class="content-pane" style="display:none;">
      </div>
      <div id="vvf7" class="content-pane" style="display:none;">
      </div>
      <div id="vvf8" class="content-pane" style="display:none;">
      </div>
      
      <!-- Macroeconomic Content Panes -->
      <div id="macro1" class="content-pane" style="display:none;">
        <div>{gdp_consensus_html}</div>
        <div>{lei_z_table}</div>
      </div>
      <div id="macro2" class="content-pane" style="display:none;">
        {manufac}
      </div>
      <div id="macro3" class="content-pane" style="display:none;">
        {eco_surpris_df_html}
      </div>
      <div id="macro4" class="content-pane" style="display:none;">
        {deficit_matrix}
      </div>
      <div id="macro5" class="content-pane" style="display:none;">
      </div>
      <div id="macro6" class="content-pane" style="display:none;">
        {corr_chart}
      </div>
      
      <!-- Technicals / Sentiment / Risk Aversion Content Panes -->
      <div id="tech1" class="content-pane" style="display:none;">
        {technicals_graphs_html}
      </div>
      <div id="tech2" class="content-pane" style="display:none;">
      </div>
      <div id="tech3" class="content-pane" style="display:none;">
      </div>
      <div id="tech4" class="content-pane" style="display:none;">
      </div>
      <div id="tech5" class="content-pane" style="display:none;">
      </div>
      
      <!-- Bond Futures and Yield Curve Content Panes -->
      <div id="misc1" class="content-pane" style="display:none;">
        <div>{rate_futures_html}</div>
        <div>{ten_10y_decomp_html}</div>  
      </div>
      <div id="misc2" class="content-pane" style="display:none;">
        <div>{US_chart_html}</div>
        <div>{AU_chart_html}</div>
        <div>{EU_chart_html}</div>
        <div>{Globalhedged_chart_html}</div>
        <div>{Global_chart_html}</div>
      </div>
      <div id="misc3" class="content-pane" style="display:none;">
        <div>{Aus_comp_chart_html}</div>
        <div>{Aus_cred_chart_html}</div>
        <div>{Aus_FRN_chart_html}</div>
      </div>
      <div id="misc4" class="content-pane" style="display:none;">
        <div>{US_corp_chart_html}</div>
        <div>{US_cred_chart_html}</div>
      </div>
      <div id="misc5" class="content-pane" style="display:none;">
        {tips_html}
      </div>
    </div>
  </div>
  
  <!-- jQuery and Bootstrap JS -->
  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> <!-- Add Plotly CDN here -->
  <script>

    function showValuationDropdown(metricId, ids) {{
      ids.forEach(id => {{
        const el = document.getElementById(id);
        if (el) el.style.display = "none";
      }});

      const selected = document.getElementById(metricId);
      const defaultMsg = document.getElementById("valuation-default");
      if (selected) {{
        selected.style.display = "block";
        if (defaultMsg) defaultMsg.style.display = "none";
      }} else {{
        if (defaultMsg) defaultMsg.style.display = "block";
      }}

      if (selected) {{
        setTimeout(() => {{
          const graphs = selected.querySelectorAll('.js-plotly-plot');
          graphs.forEach(graph => {{
            if (typeof Plotly !== 'undefined') {{
              Plotly.Plots.resize(graph);
            }}
          }});
        }}, 600);
      }}
    }}

    function showAusValuation(metricId) {{
      showValuationDropdown(metricId, [
        "aus_forward_pe", "aus_cape", "aus_price_to_book", "aus_price_to_sales", "aus_valuation_composite"
      ]);
    }}

    function reigAusValuation(metricId) {{
      showValuationDropdown(metricId, [
        "_forward_pe", "_cape", "_price_to_book", "_price_to_sales", "_valuation_composite"
      ]);
    }}

    function showValuation(metricId) {{
      showValuationDropdown(metricId, [
        "factor_forward_pe", "factor_cape", "factor_price_to_book", "factor_price_to_sales", "factor_valuation_composite"
      ]);
    }}


  // Function to display the corresponding subtab menu based on the selected main tab
  function showMainTab(tabId) {{
    // Hide all subtabs first
    var subtabs = document.getElementsByClassName('subtabs');
    for (var i = 0; i < subtabs.length; i++) {{
      subtabs[i].style.display = 'none';
    }}

    // Show current tab’s subtab section
    var currentSubtab = document.getElementById('subtabs-' + tabId);
    if (currentSubtab) {{
      currentSubtab.style.display = 'block';
    }}

    // Automatically open the first subtab under the selected main tab
    const firstSubtab = document.querySelector(`#subtabs-${{tabId}} .list-group-item`);
    if (firstSubtab && firstSubtab.getAttribute('onclick')) {{
      const onclickContentId = firstSubtab.getAttribute('onclick').match(/showContent\\('(.+?)'\\)/);
      if (onclickContentId && onclickContentId[1]) {{
        showContent(onclickContentId[1]);
      }}
    }}

    // Tab highlighting logic
    var mainTabLinks = document.querySelectorAll('.nav-tabs .nav-link');
    for (var i = 0; i < mainTabLinks.length; i++) {{
      mainTabLinks[i].classList.remove('active');
    }}

    var activeLink = document.querySelector('.nav-tabs .nav-link[data-main-tab="{{' + tabId + '}}"]');
    if (activeLink) {{
      activeLink.classList.add('active');
    }}
  }}

  document.addEventListener("DOMContentLoaded", function () {{
    showMainTab('innova');
    showContent('landing');
  }});

  
  function showContent(contentId) {{
    var panes = document.getElementsByClassName('content-pane');
    for (var i = 0; i < panes.length; i++) {{
      panes[i].style.display = 'none';
    }}

    var el = document.getElementById(contentId);
    if (el) {{
      el.style.display = 'block';

      // Resize Plotly charts after showing the content
      setTimeout(function () {{
        var plotlyGraphs = el.querySelectorAll('.js-plotly-plot');
        for (var j = 0; j < plotlyGraphs.length; j++) {{
          if (typeof Plotly !== 'undefined') {{
            Plotly.Plots.resize(plotlyGraphs[j]);
          }}
        }}
      }}, 600);
    }}
  }}

</script>
  
</body>
</html>
"""
# Write the HTML to a file
with open("index.html", "w", encoding='utf-8') as file:
    file.write(html_template)